# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
from keras.utils import pad_sequences

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          1000000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5)                 280       
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/Users/itnas/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [11]:
import tensorflow as tf
with tf.device("/cpu:0"):
    model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-22 09:09:52.990672: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-22 09:09:53.420663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


781/782 [============================>.] - ETA: 0s - loss: 0.6615 - accuracy: 0.6189

2023-03-22 09:10:08.946211: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 25s 31ms/step - loss: 0.6615 - accuracy: 0.6189 - val_loss: 0.6044 - val_accuracy: 0.6915
Epoch 2/10
782/782 [==============================] - 23s 29ms/step - loss: 0.5513 - accuracy: 0.7264 - val_loss: 0.5322 - val_accuracy: 0.7307
Epoch 3/10
782/782 [==============================] - 22s 29ms/step - loss: 0.4840 - accuracy: 0.7686 - val_loss: 0.4966 - val_accuracy: 0.7553
Epoch 4/10
782/782 [==============================] - 22s 29ms/step - loss: 0.4439 - accuracy: 0.7946 - val_loss: 0.4767 - val_accuracy: 0.7673
Epoch 5/10
782/782 [==============================] - 22s 29ms/step - loss: 0.4163 - accuracy: 0.8103 - val_loss: 0.4663 - val_accuracy: 0.7737
Epoch 6/10
782/782 [==============================] - 22s 29ms/step - loss: 0.3976 - accuracy: 0.8213 - val_loss: 0.4532 - val_accuracy: 0.7817
Epoch 7/10
782/782 [==============================] - 22s 29ms/step - loss: 0.3832 - accuracy: 0.8278 - val_loss: 0.4517 - val_accuracy: 0.7846
Epo

In [13]:
with tf.device("/cpu:0"):
    score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 9s 11ms/step - loss: 0.4466 - accuracy: 0.7894
Test score: 0.44658035039901733
Test accuracy: 0.789359986782074


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [14]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [15]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [16]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/Users/itnas/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [17]:
with tf.device("/cpu:0"):
    model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-22 09:19:01.088003: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.5970 - accuracy: 0.6749

2023-03-22 09:19:33.683675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 53s 67ms/step - loss: 0.5970 - accuracy: 0.6749 - val_loss: 0.5045 - val_accuracy: 0.7564
Epoch 2/10
782/782 [==============================] - 54s 69ms/step - loss: 0.4397 - accuracy: 0.7982 - val_loss: 0.4696 - val_accuracy: 0.7754
Epoch 3/10
782/782 [==============================] - 52s 67ms/step - loss: 0.3723 - accuracy: 0.8366 - val_loss: 0.4029 - val_accuracy: 0.8158
Epoch 4/10
782/782 [==============================] - 49s 63ms/step - loss: 0.3322 - accuracy: 0.8605 - val_loss: 0.3854 - val_accuracy: 0.8260
Epoch 5/10
782/782 [==============================] - 48s 61ms/step - loss: 0.3047 - accuracy: 0.8724 - val_loss: 0.3841 - val_accuracy: 0.8286
Epoch 6/10
782/782 [==============================] - 47s 61ms/step - loss: 0.2858 - accuracy: 0.8814 - val_loss: 0.3605 - val_accuracy: 0.8394
Epoch 7/10
782/782 [==============================] - 48s 61ms/step - loss: 0.2723 - accuracy: 0.8870 - val_loss: 0.3654 - val_accuracy: 0.8394
Epo

In [19]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [20]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [21]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [22]:
with tf.device("/cpu:0"):
    model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-22 09:32:07.205449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.6294 - accuracy: 0.6352

2023-03-22 09:32:41.900284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 56s 70ms/step - loss: 0.6294 - accuracy: 0.6352 - val_loss: 0.5441 - val_accuracy: 0.7246
Epoch 2/10
782/782 [==============================] - 50s 64ms/step - loss: 0.4927 - accuracy: 0.7618 - val_loss: 0.4733 - val_accuracy: 0.7743
Epoch 3/10
782/782 [==============================] - 49s 63ms/step - loss: 0.4320 - accuracy: 0.7996 - val_loss: 0.4371 - val_accuracy: 0.7957
Epoch 4/10
782/782 [==============================] - 52s 67ms/step - loss: 0.3913 - accuracy: 0.8235 - val_loss: 0.4087 - val_accuracy: 0.8117
Epoch 5/10
782/782 [==============================] - 58s 74ms/step - loss: 0.3640 - accuracy: 0.8387 - val_loss: 0.3946 - val_accuracy: 0.8208
Epoch 6/10
782/782 [==============================] - 52s 67ms/step - loss: 0.3440 - accuracy: 0.8502 - val_loss: 0.3867 - val_accuracy: 0.8255
Epoch 7/10
782/782 [==============================] - 52s 67ms/step - loss: 0.3292 - accuracy: 0.8590 - val_loss: 0.3729 - val_accuracy: 0.8331
Epo

In [23]:
# Out of curiosity, run for 10 more epochs
with tf.device("/cpu:0"):
    model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 52s 67ms/step - loss: 0.2963 - accuracy: 0.8753 - val_loss: 0.3741 - val_accuracy: 0.8351
Epoch 2/10
782/782 [==============================] - 56s 71ms/step - loss: 0.2920 - accuracy: 0.8772 - val_loss: 0.3569 - val_accuracy: 0.8418
Epoch 3/10
782/782 [==============================] - 50s 64ms/step - loss: 0.2871 - accuracy: 0.8794 - val_loss: 0.3618 - val_accuracy: 0.8413
Epoch 4/10
782/782 [==============================] - 50s 64ms/step - loss: 0.2837 - accuracy: 0.8806 - val_loss: 0.3694 - val_accuracy: 0.8399
Epoch 5/10
782/782 [==============================] - 51s 66ms/step - loss: 0.2803 - accuracy: 0.8830 - val_loss: 0.3544 - val_accuracy: 0.8440
Epoch 6/10
782/782 [==============================] - 52s 66ms/step - loss: 0.2782 - accuracy: 0.8820 - val_loss: 0.3582 - val_accuracy: 0.8440
Epoch 7/10
782/782 [==============================] - 52s 67ms/step - loss: 0.2756 - accuracy: 0.8842 - val_loss: 0.3639 - val_accuracy:

In [28]:
with tf.device("/cpu:0"):
    print(model_rnn.predict(x_test)>0.5)

782/782 [==============================] - 20s 26ms/step
[[False]
 [ True]
 [ True]
 ...
 [False]
 [False]
 [ True]]


In [29]:
with tf.device("/cpu:0"):
    print(model_rnn.predict(x_test))

782/782 [==============================] - 21s 26ms/step
[[0.0484817 ]
 [0.9748107 ]
 [0.901     ]
 ...
 [0.01542827]
 [0.04022381]
 [0.92631656]]


In [27]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

---
### Machine Learning Foundation (C) 2020 IBM Corporation